In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

import plotly.express as px

## Image Text Retrieval

In [2]:
uniform_results_path = '/fs/cfar-projects/low-bit-vision/uniform_results/image_text_retrieval/blip2_flickr_results.csv'
df_uniform = pd.read_csv(uniform_results_path)
df_uniform = df_uniform.dropna(axis = 1, how = 'all')

# with pd.option_context('display.max_rows', 10):
#     display(df_uniform)


In [3]:
awq_results_path = '/fs/cfar-projects/low-bit-vision/awq_results/image_text_retrieval/awq_image_text_retrieval.csv'
df_awq = pd.read_csv(awq_results_path)
# bits --> megabytes
df_awq['model_size'] = df_awq['model_size'] / 8e6
df_awq[:5]

,vit_bits,qformer_bits,txt_r1,txt_r5,txt_r10,txt_r_mean,img_r1,img_r5,img_r10,img_r_mean,r_mean,agg_metrics,model_size
0,2,2,65.6,85.1,91.9,80.866667,55.66,78.76,85.30,73.240000,77.053333,80.866667,387.970088
1,2,3,78.4,93.3,96.6,89.433333,65.12,85.56,90.52,80.400000,84.916667,89.433333,408.189992
2,2,4,78.2,93.8,96.4,89.466667,64.74,84.86,89.72,79.773333,84.620000,89.466667,428.409896
3,2,5,79.1,93.1,95.9,89.366667,64.52,84.26,89.34,79.373333,84.370000,89.366667,448.629800
4,2,6,78.5,93.6,95.7,89.266667,64.40,84.32,89.40,79.373333,84.320000,89.266667,468.849704


In [7]:
fig = px.scatter_3d(df_awq, x='img_r1', y="txt_r1", z = 'model_size',
                    color = 'vit_bits',
                    symbol = 'qformer_bits',
                    hover_data = df_awq.columns,
                    title = 'AWQ Blip-2 Flickr30k Information Retrieval',)


fig.update_traces(marker=dict(size=4.5))

fig.update_layout(scene = dict(
                  xaxis_title='Text-->Img R@1',
                  yaxis_title='Img -->Text R@1',
                  zaxis_title='Model Size (MB)',
                  
                  xaxis = dict(autorange='reversed'),
                  yaxis = dict(autorange='reversed')),)

fig.write_html("awq_retrieval.html")

# fig.show()